In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

print(interactions_df['room_segment'].isnull().values.any())

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[0-160],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[160-260],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[160-260],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [81]:
def prepare_users_df_2(interactions_df):

    users_df = interactions_df.copy()
    #https://www.sharpsightlabs.com/blog/pandas-get-dummies/
    users_df = pd.get_dummies(users_df)
    users_df = users_df.drop(columns=['item_id'])
    users_df = users_df.set_index('user_id')
    users_df = users_df.groupby('user_id').sum()
#     users_df = users_df.apply(lambda x: np.log(x+1))
    users_df = users_df.div(users_df.sum(axis=1), axis=0)
    users_df = users_df.add_prefix('user_')
    users_df = users_df.reset_index()
    user_features = list(users_df.columns)
    user_features.remove('user_id')
    return users_df, user_features

def prepare_users_df(interactions_df):

    users_df = interactions_df.copy()
    #https://www.sharpsightlabs.com/blog/pandas-get-dummies/
    users_df = pd.get_dummies(users_df)
    users_df = users_df.drop(columns=['item_id'])
#     users_df = users_df.set_index('user_id')
#     users_df = users_df.add_prefix('user_')
#     users_df = users_df.reset_index()
    user_features = list(users_df.columns)
    user_features.remove('user_id')
    users_df = users_df.drop_duplicates()
    return users_df, user_features

    

# users_df, user_features = prepare_users_df(interactions_df)

# print(user_features)

# display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

def prepare_users_df_3(interactions_df):
    users_df = interactions_df.copy()
    users_df = users_df.drop(columns=['item_id'])
    
    users_df.loc[users_df['term'] == 'WinterVacation', 'user_term'] = 1
    users_df.loc[users_df['term'] == 'Easter', 'user_term'] = 2
    users_df.loc[users_df['term'] == 'OffSeason', 'user_term'] = 3
    users_df.loc[users_df['term'] == 'HighSeason', 'user_term'] = 4
    users_df.loc[users_df['term'] == 'LowSeason', 'user_term'] = 5
    users_df.loc[users_df['term'] == 'MayLongWeekend', 'user_term'] = 6
    users_df.loc[users_df['term'] == 'NewYear', 'user_term'] = 7
    users_df.loc[users_df['term'] == 'Christmas', 'user_term'] = 8
    users_df.loc[users_df['length_of_stay_bucket'] == '[0-1]', 'user_length_of_stay_bucket'] = 1
    users_df.loc[users_df['length_of_stay_bucket'] == '[2-3]', 'user_length_of_stay_bucket'] = 2
    users_df.loc[users_df['length_of_stay_bucket'] == '[4-7]', 'user_length_of_stay_bucket'] = 3
    users_df.loc[users_df['length_of_stay_bucket'] == '[8-inf]', 'user_length_of_stay_bucket'] = 4
    users_df.loc[users_df['rate_plan'] == 'Standard', 'user_rate_plan'] = 1
    users_df.loc[users_df['rate_plan'] == 'Nonref', 'user_rate_plan'] = 2
    users_df.loc[users_df['room_segment'] == '[0-160]', 'user_room_segment'] = 1
    users_df.loc[users_df['room_segment'] == '[160-260]', 'user_room_segment'] = 2
    users_df.loc[users_df['room_segment'] == '[260-360]', 'user_room_segment'] = 3
    users_df.loc[users_df['room_segment'] == '[360-500]', 'user_room_segment'] = 4
    users_df.loc[users_df['room_segment'] == '[500-900]', 'user_room_segment'] = 5
    users_df.loc[users_df['n_people_bucket'] == '[1-1]', 'user_n_people_bucket'] = 1
    users_df.loc[users_df['n_people_bucket'] == '[2-2]', 'user_n_people_bucket'] = 2
    users_df.loc[users_df['n_people_bucket'] == '[3-4]', 'user_n_people_bucket'] = 3
    users_df.loc[users_df['n_people_bucket'] == '[5-inf]', 'user_n_people_bucket'] = 4
    users_df.loc[users_df['weekend_stay'] == 'True', 'user_weekend_stay'] = 1
    users_df.loc[users_df['weekend_stay'] == 'False', 'user_weekend_stay'] = 2
    users_df = users_df.drop(columns=['term'])
    users_df = users_df.drop(columns=['length_of_stay_bucket'])
    users_df = users_df.drop(columns=['rate_plan'])
    users_df = users_df.drop(columns=['room_segment'])
    users_df = users_df.drop(columns=['n_people_bucket'])
    users_df = users_df.drop(columns=['weekend_stay'])
    users_df = users_df.set_index('user_id')
    users_df = users_df.groupby('user_id').mean()
    users_df = users_df.apply(lambda x: np.log(x+1))
    users_df = users_df.div(users_df.sum(axis=1), axis=0)
    users_df = users_df.reset_index()
    user_features = list(users_df.columns)
    user_features.remove('user_id')
    return users_df.fillna(0), user_features
# users_df_2, user_features_2 = prepare_users_df_2(interactions_df)

# print(user_features_2)
users_df, user_features = prepare_users_df(interactions_df)
print(user_features)
print(len(users_df))
display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))


['term_WinterVacation', 'term_Easter', 'term_OffSeason', 'term_HighSeason', 'term_LowSeason', 'term_MayLongWeekend', 'term_NewYear', 'term_Christmas', 'length_of_stay_bucket_[0-1]', 'length_of_stay_bucket_[2-3]', 'length_of_stay_bucket_[4-7]', 'length_of_stay_bucket_[8-inf]', 'rate_plan_Standard', 'rate_plan_Nonref', 'room_segment_[0-160]', 'room_segment_[160-260]', 'room_segment_[260-360]', 'room_segment_[360-500]', 'room_segment_[500-900]', 'n_people_bucket_[1-1]', 'n_people_bucket_[2-2]', 'n_people_bucket_[3-4]', 'n_people_bucket_[5-inf]', 'weekend_stay_True', 'weekend_stay_False']
15054


,user_id,term_WinterVacation,term_Easter,term_OffSeason,term_HighSeason,term_LowSeason,term_MayLongWeekend,term_NewYear,term_Christmas,length_of_stay_bucket_[0-1],...,room_segment_[160-260],room_segment_[260-360],room_segment_[360-500],room_segment_[500-900],n_people_bucket_[1-1],n_people_bucket_[2-2],n_people_bucket_[3-4],n_people_bucket_[5-inf],weekend_stay_True,weekend_stay_False
0,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
42,50,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
47,50,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
83,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
318,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
333,706,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
387,706,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
439,706,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
456,706,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
515,706,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [82]:
def prepare_items_df_2(interactions_df):
    
    items_df = interactions_df.copy()
    #https://www.sharpsightlabs.com/blog/pandas-get-dummies/
    if 'user_id' in items_df.columns:
        items_df = items_df.drop(columns=['user_id'])
    items_df = pd.get_dummies(items_df)  
    items_df = items_df.set_index('item_id')
    items_df = items_df.groupby('item_id').sum()
#     items_df = items_df.apply(lambda x: np.log(x+1))
#     items_df = items_df.div(items_df.sum(axis=1), axis=0)
#     items_df = items_df.add_prefix('item_')
#     items_df = items_df.reset_index()
    item_features = list(items_df.columns)
    item_features.remove('item_id')
    
    return items_df, item_features

def prepare_items_df(interactions_df):
    
    items_df = interactions_df.copy()
    #https://www.sharpsightlabs.com/blog/pandas-get-dummies/
    if 'user_id' in items_df.columns:
        items_df = items_df.drop(columns=['user_id'])
    items_df = pd.get_dummies(items_df)  
    items_df = items_df.set_index('item_id')
    items_df = items_df.add_prefix('item_')
    items_df = items_df.reset_index()
    item_features = list(items_df.columns)
    item_features.remove('item_id')
    items_df = items_df.drop_duplicates()
    return items_df, item_features


# items_df, item_features = prepare_items_df(interactions_df)

# print(item_features)

# display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

def prepare_items_df_3(interactions_df):
    items_df = interactions_df.copy()
    if 'user_id' in items_df.columns:
        items_df = items_df.drop(columns=['user_id'])
    
    items_df.loc[items_df['term'] == 'WinterVacation', 'item_term'] = 1
    items_df.loc[items_df['term'] == 'Easter', 'item_term'] = 2
    items_df.loc[items_df['term'] == 'OffSeason', 'item_term'] = 3
    items_df.loc[items_df['term'] == 'HighSeason', 'item_term'] = 4
    items_df.loc[items_df['term'] == 'LowSeason', 'item_term'] = 5
    items_df.loc[items_df['term'] == 'MayLongWeekend', 'item_term'] = 6
    items_df.loc[items_df['term'] == 'NewYear', 'item_term'] = 7
    items_df.loc[items_df['term'] == 'Christmas', 'item_term'] = 8
    items_df.loc[items_df['length_of_stay_bucket'] == '[0-1]', 'item_length_of_stay_bucket'] = 1
    items_df.loc[items_df['length_of_stay_bucket'] == '[2-3]', 'item_length_of_stay_bucket'] = 2
    items_df.loc[items_df['length_of_stay_bucket'] == '[4-7]', 'item_length_of_stay_bucket'] = 3
    items_df.loc[items_df['length_of_stay_bucket'] == '[8-inf]', 'item_length_of_stay_bucket'] = 4
    items_df.loc[items_df['rate_plan'] == 'Standard', 'item_rate_plan'] = 1
    items_df.loc[items_df['rate_plan'] == 'Nonref', 'item_rate_plan'] = 2
    items_df.loc[items_df['room_segment'] == '[0-160]', 'item_room_segment'] = 1
    items_df.loc[items_df['room_segment'] == '[160-260]', 'item_room_segment'] = 2
    items_df.loc[items_df['room_segment'] == '[260-360]', 'item_room_segment'] = 3
    items_df.loc[items_df['room_segment'] == '[360-500]', 'item_room_segment'] = 4
    items_df.loc[items_df['room_segment'] == '[500-900]', 'item_room_segment'] = 5
    items_df.loc[items_df['n_people_bucket'] == '[1-1]', 'item_n_people_bucket'] = 1
    items_df.loc[items_df['n_people_bucket'] == '[2-2]', 'item_n_people_bucket'] = 2
    items_df.loc[items_df['n_people_bucket'] == '[3-4]', 'item_n_people_bucket'] = 3
    items_df.loc[items_df['n_people_bucket'] == '[5-inf]', 'item_n_people_bucket'] = 4
    items_df.loc[items_df['weekend_stay'] == 'True', 'item_weekend_stay'] = 1
    items_df.loc[items_df['weekend_stay'] == 'False', 'item_weekend_stay'] = 2
    items_df = items_df.drop(columns=['term'])
    items_df = items_df.drop(columns=['length_of_stay_bucket'])
    items_df = items_df.drop(columns=['rate_plan'])
    items_df = items_df.drop(columns=['room_segment'])
    items_df = items_df.drop(columns=['n_people_bucket'])
    items_df = items_df.drop(columns=['weekend_stay'])
    items_df = items_df.set_index('item_id')
    items_df = items_df.groupby('item_id').mean()
    items_df = items_df.apply(lambda x: np.log(x+1))
    items_df = items_df.div(items_df.sum(axis=1), axis=0)
    items_df = items_df.reset_index()
    item_features = list(items_df.columns)
    item_features.remove('item_id')
#     display(items_df)
    return items_df.fillna(0), item_features


items_df, item_features = prepare_items_df(interactions_df)

print(item_features)
print(len(items_df))
display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))


['item_term_WinterVacation', 'item_term_Easter', 'item_term_OffSeason', 'item_term_HighSeason', 'item_term_LowSeason', 'item_term_MayLongWeekend', 'item_term_NewYear', 'item_term_Christmas', 'item_length_of_stay_bucket_[0-1]', 'item_length_of_stay_bucket_[2-3]', 'item_length_of_stay_bucket_[4-7]', 'item_length_of_stay_bucket_[8-inf]', 'item_rate_plan_Standard', 'item_rate_plan_Nonref', 'item_room_segment_[0-160]', 'item_room_segment_[160-260]', 'item_room_segment_[260-360]', 'item_room_segment_[360-500]', 'item_room_segment_[500-900]', 'item_n_people_bucket_[1-1]', 'item_n_people_bucket_[2-2]', 'item_n_people_bucket_[3-4]', 'item_n_people_bucket_[5-inf]', 'item_weekend_stay_True', 'item_weekend_stay_False']
1063


,item_id,item_term_WinterVacation,item_term_Easter,item_term_OffSeason,item_term_HighSeason,item_term_LowSeason,item_term_MayLongWeekend,item_term_NewYear,item_term_Christmas,item_length_of_stay_bucket_[0-1],...,item_room_segment_[160-260],item_room_segment_[260-360],item_room_segment_[360-500],item_room_segment_[500-900],item_n_people_bucket_[1-1],item_n_people_bucket_[2-2],item_n_people_bucket_[3-4],item_n_people_bucket_[5-inf],item_weekend_stay_True,item_weekend_stay_False
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
1,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
2,2,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
3,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5,5,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
6,6,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [83]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
#         self.model = Ridge()
#         self.model = Lasso(alpha=1.0)
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)

#         items_df = items_df.loc[:, ['item_id'] + item_features]
        items_df = items_df.loc[:, item_features]
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []
        
        ########################
        # Write your code here #
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.
        ##
        min_item_id = interactions_df['item_id'].min()
        max_item_id = interactions_df['item_id'].max()
        
        dict_interactions_df = interactions_df.to_dict('records')
        for row in dict_interactions_df:
            #filter on user_id
            filtered_df_on_user_id = interactions_df[interactions_df.user_id.isin([row['user_id']])]
            
            #find five negative interactions
            nr_of_neg_interactions = 0
            while True:
                if nr_of_neg_interactions == 5:
                    break
                
                rand_item_id = np.random.randint(min_item_id, max_item_id)
                if not (filtered_df_on_user_id['item_id'].values == [rand_item_id]).any():
                    negative_interactions.append((row['user_id'], rand_item_id, 0))
                    nr_of_neg_interactions += 1
        
        #generator of negative interactions is taken about 11-18 seconds
        ##
        
        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])

        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        #change NAN with fillna
#         interactions_df = interactions_df.fillna(0).astype(int)
        #
    
        ##sample data
#         interactions_df = interactions_df.sample(frac=0.05)
        ##
        display(interactions_df.loc[:, user_features + item_features].columns)
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values
    
        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        ########################
        # Write your code here #
        # Prepare users_df and items_df
        # For users_df you need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations).
        # Note that for users who were not in the hotel before (which is true for most users)
        # there will be no features in self.users_df. For such users you can initialize their features
        # with all zeros (for instance with fillna(0)), but you can also try to use average feature
        # values from self.users_df (this way you would trear a new user as an average user).
        # For items you have to apply the prepare_items_df method to items_df.
        ##
        #users_df
        users_df = users_df.merge(self.users_df, how='left', on='user_id')

        users_df = users_df.fillna(0).astype(int)
        
        #items_df
        items_df, item_features = prepare_items_df(items_df) 
        ##
        
        # Score the items
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            
            ########################
            # Write your code here #
            # Create a Carthesian product of users from users_df and items from items_df
            ##
            only_one_user_df = users_df.loc[users_df['user_id'].isin([user['user_id']])]
            cross_df = only_one_user_df.merge(items_df, how="cross")
            cross_df = cross_df.drop_duplicates()
            display(cross_df.columns)
            ##

            ########################
            # Write your code here #
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            ##
            scores = self.model.predict(cross_df.values)
            
            cross_df['scores'] = pd.Series(scores)
            ##
            
            ########################
            # Write your code here #
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            chosen_ids = np.argsort(-scores)[:n_recommendations]
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': cross_df.loc[cross_df['scores'] == scores[item_id], 'item_id'].iloc[0],#item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df

# cb_user_item_recommender = ContentBasedUserItemRecommender()
# cb_user_item_recommender.fit(interactions_df, None, None)
# cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [84]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [85]:
# Fit method
import time
t0 = time.time()
cb_user_item_recommender = ContentBasedUserItemRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)
print('Total fit time: {}'.format(time.time() - t0))

KeyError: 'item_id'

In [ ]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations.head(60))

# Tuning method

In [9]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [10]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [55]:
param_space = {
    'n_neg_per_pos': hp.choice('n_neg_per_pos', np.arange(1, 10, dtype=int))
}
 
best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 10/10 [59:33<00:00, 357.30s/trial, best loss: -0.004655099449851883]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.0,0.0,0.000339,0.003055,0.0,0.0,0.000131,0.000933


Best parameters:
{'n_neg_per_pos': 0}


In [61]:
param_space = {
#     'n_neg_per_pos': hp.choice('n_neg_per_pos', np.arange(1, 10, dtype=int))
    'solver': hp.choice('liblinear', 'newton-cg', 'saga')
}
 
best_param_set = tune_recommender(ContentBasedUserItemRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 10/10 [58:41<00:00, 352.14s/trial, best loss: -0.10918324164198054]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,ContentBasedUserItemRecommender,0.010523,0.051256,0.06721,0.120502,0.010523,0.0312,0.038071,0.054634


Best parameters:
{'n_neg_per_pos': 3}


In [ ]:
param_space = {
    'n_neg_per_pos': hp.choice('n_neg_per_pos', np.arange(1, 10, dtype=int)),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

In [67]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: 'float' object cannot be interpreted as an integer



  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]
Best parameters:
None


Traceback (most recent call last):
  File "<ipython-input-14-0fde002c7dd7>", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 540, in fmin
    return trials.fmin(
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/base.py", line 671, in fmin
    return fmin(
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 586, in fmin
    rval.exhaust()
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 364, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 300, in run
    self.serial_evaluate()
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 178, in serial_evaluate
    result = self.domain.evaluate(spec, ctrl)
  File "/home/jbiesek/.local/lib/python3.

In [18]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|          | 0/300 [00:00<?, ?trial/s, best loss=?]

job exception: 'float' object cannot be interpreted as an integer



  0%|          | 0/300 [00:00<?, ?trial/s, best loss=?]
Best parameters:
None


Traceback (most recent call last):
  File "<ipython-input-14-0fde002c7dd7>", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 540, in fmin
    return trials.fmin(
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/base.py", line 671, in fmin
    return fmin(
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 586, in fmin
    rval.exhaust()
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 364, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 300, in run
    self.serial_evaluate()
  File "/home/jbiesek/.local/lib/python3.8/site-packages/hyperopt/fmin.py", line 178, in serial_evaluate
    result = self.domain.evaluate(spec, ctrl)
  File "/home/jbiesek/.local/lib/python3.

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [111]:
import time
t0 = time.time()
cb_user_item_recommender = ContentBasedUserItemRecommender(
    **{'n_neg_per_pos': 5})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LogisticRegressionRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)
print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LogisticRegressionRecommender,0.000339,0.039375,0.075696,0.100815,0.000339,0.019946,0.035589,0.043033


Total evaluation time: 495.614280462265


In [73]:
from recommenders.amazon_recommender import AmazonRecommender
t0 = time.time()
amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)
print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.035981,0.09165,0.132043,0.185336,0.035981,0.068082,0.084824,0.102078


Total evaluation time: 247.69475984573364


In [74]:
tts_results = pd.concat([linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LogisticRegressionRecommender,0.010523,0.051256,0.067549,0.120502,0.010523,0.035911,0.042914,0.060862
1,AmazonRecommender,0.035981,0.091650,0.132043,0.185336,0.035981,0.068082,0.084824,0.102078
